Dữ liệu được lấy từ: https://github.com/ds4v/vietnamese-pos-tagging

In [1]:
from collections import defaultdict
import unicodedata as ud
import numpy as np
import math
import ast
import re

### Thu thập dữ liệu

In [2]:
sentences = open('dataset/sentences.txt', encoding='utf-8').readlines()
tokenize_sentences = [sentence.split(' ') for sentence in sentences]

In [3]:
print('Số lượng câu đã thu thập:', len(sentences))
sentences[0:2]

Số lượng câu đã thu thập: 60


['Pha lập công trên đã giúp Rashford giải hạn bàn thắng tại sân Old Trafford kéo dài 845 phút .\n',
 'Với 3 điểm có được trong trận đấu này , Quỷ đỏ đã leo lên vị trí thứ 2 trên bảng xếp hạng Premier League với 30 điểm , chỉ kém đội đầu bảng Liverpool 2 điểm .\n']

In [4]:
max_str = max(tokenize_sentences, key=len)
print('Câu có số từ nhiều nhât:', len(max_str))
' '.join(max_str)

Câu có số từ nhiều nhât: 46


'Những thực phẩm được chế biến như hun khói , thức ăn ngâm tẩm , muối , món ăn chứa lượng muối cao thường có tỷ lệ mắc ung thư dạ dày cao hơn những người có thói quen ăn uống nhạt và thanh đạm .\n'

In [5]:
min_str = min(tokenize_sentences, key=len)
print('Câu có số từ ít nhât:', len(min_str))
' '.join(min_str)

Câu có số từ ít nhât: 8


'Nhiều người có hoàn cảnh giống ông .\n'

### Tách từ

In [6]:
def syllablize(sentence): # Tách âm tiết cho một câu tiếng Việt
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

In [7]:
syllables = syllablize(sentences[1])
syllables

['Với',
 '3',
 'điểm',
 'có',
 'được',
 'trong',
 'trận',
 'đấu',
 'này',
 ',',
 'Quỷ',
 'đỏ',
 'đã',
 'leo',
 'lên',
 'vị',
 'trí',
 'thứ',
 '2',
 'trên',
 'bảng',
 'xếp',
 'hạng',
 'Premier',
 'League',
 'với',
 '30',
 'điểm',
 ',',
 'chỉ',
 'kém',
 'đội',
 'đầu',
 'bảng',
 'Liverpool',
 '2',
 'điểm',
 '.']

In [8]:
# Tải từ trong vi-vocab.txt
with open('data/dic3.txt', encoding='utf8') as f:
    vocab = f.read().split('\n')
# Xây dựng từ điển vocabs theo độ dài từ
vocabs = defaultdict(list)
for word in vocab:
    vocabs[len(word.split())].append(word)

print('Số lượng từ ghép và cụm từ trong vocab:', len(vocab))
print('Số bộ vocab phân theo độ dài:', len(vocabs))

Số lượng từ ghép và cụm từ trong vocab: 112343
Số bộ vocab phân theo độ dài: 20


In [9]:
def longest_matching(sentence, vocabs):
  words = syllablize(sentence) # tách âm tiết cho câu
  result = []
  i = len(words)-1 # index của từ hiện tại
  while i > -1: 
    word = '' 
    # tìm kiếm trong từ điển theo chiều dài của từ ưu tiên từ dài trước
    for j in range(i+1):
      ls_word = words[j:i+1]
      word = ' '.join(ls_word)
      # xem thử có trong từ điển không
      if word.lower() in vocabs[len(ls_word)]:
        i = j
        break
    result = [word] + result
    i-=1
  return result # return the final list

In [10]:
for sentence in sentences[0:5]:
  print('Câu:', sentence)
  print('Tách từ:', syllablize(sentence))
  print('Tách từ dài nhất:', longest_matching(sentence, vocabs))
  print('---------------------------------------------------')

Câu: Pha lập công trên đã giúp Rashford giải hạn bàn thắng tại sân Old Trafford kéo dài 845 phút .

Tách từ: ['Pha', 'lập', 'công', 'trên', 'đã', 'giúp', 'Rashford', 'giải', 'hạn', 'bàn', 'thắng', 'tại', 'sân', 'Old', 'Trafford', 'kéo', 'dài', '845', 'phút', '.']
Tách từ dài nhất: ['Pha', 'lập công', 'trên', 'đã', 'giúp', 'Rashford', 'giải hạn', 'bàn thắng', 'tại', 'sân', 'Old', 'Trafford', 'kéo dài', '845', 'phút', '.']
---------------------------------------------------
Câu: Với 3 điểm có được trong trận đấu này , Quỷ đỏ đã leo lên vị trí thứ 2 trên bảng xếp hạng Premier League với 30 điểm , chỉ kém đội đầu bảng Liverpool 2 điểm .

Tách từ: ['Với', '3', 'điểm', 'có', 'được', 'trong', 'trận', 'đấu', 'này', ',', 'Quỷ', 'đỏ', 'đã', 'leo', 'lên', 'vị', 'trí', 'thứ', '2', 'trên', 'bảng', 'xếp', 'hạng', 'Premier', 'League', 'với', '30', 'điểm', ',', 'chỉ', 'kém', 'đội', 'đầu', 'bảng', 'Liverpool', '2', 'điểm', '.']
Tách từ dài nhất: ['Với', '3', 'điểm', 'có được', 'trong', 'trận đấu', 'này

In [11]:
longest_matching('nhưng sự thực hiện vẫn còn chưa phù hợp', vocabs)

['nhưng', 'sự thực hiện', 'vẫn còn', 'chưa', 'phù hợp']

In [12]:
def tokenize_sentences(sentence):
    return ' '.join([x.replace(' ','_') for x in longest_matching(sentence, vocabs)])

tokenize_sentences(sentences[0])

'Pha lập_công trên đã giúp Rashford giải_hạn bàn_thắng tại sân Old Trafford kéo_dài 845 phút .'

In [13]:
longest_matching_sentences = []
for sentence in sentences:
    longest_matching_sentences.append(tokenize_sentences(sentence))
longest_matching_sentences[0:3]

['Pha lập_công trên đã giúp Rashford giải_hạn bàn_thắng tại sân Old Trafford kéo_dài 845 phút .',
 'Với 3 điểm có_được trong trận_đấu này , Quỷ đỏ đã leo_lên vị_trí thứ_2 trên bảng_xếp_hạng Premier League với 30 điểm , chỉ kém đội đầu_bảng Liverpool 2 điểm .',
 'Tổng_thống đắc_cử Joe Biden được cho_đang cân_nhắc việc_cắt_bỏ chương_trình hiện_đại hóa hạt_nhân trị_giá 1 . 000 tỷ USD do chính_quyền Tổng_thống đương_nhiệm Donald Trump đề_xuất .']

In [14]:
longest_matching_compounds = []
for sentence in longest_matching_sentences:
    for word in sentence.split():
        if '_' in word: longest_matching_compounds.append(word.lower())

print('Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching:', len(longest_matching_compounds))

Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching: 340


### Tách từ thủ công bằng tay

In [15]:
with open('tokenize/manual_tokens.txt', 'r', encoding='utf-8') as f:
    manual_tokenize_sentences = []
    sentence = ''
    for word in f:
        if word == '\n': 
            manual_tokenize_sentences.append(sentence.strip())
            sentence = ''
        else: sentence += word.replace('\n', ' ')
manual_tokenize_sentences[0:3]

['Pha lập_công trên đã giúp Rashford giải_hạn bàn_thắng tại sân Old_Trafford kéo_dài 845 phút .',
 'Với 3 điểm có được trong trận_đấu này , Quỷ_đỏ đã leo_lên vị_trí thứ 2 trên bảng xếp_hạng Premier_League với 30 điểm , chỉ kém đội đầu_bảng Liverpool 2 điểm .',
 'Tổng_thống đắc_cử Joe_Biden được cho đang cân_nhắc việc_cắt_bỏ chương_trình hiện_đại_hoá hạt_nhân trị_giá 1.000 tỷ USD do chính_quyền Tổng_thống đương_nhiệm Donald_Trump đề_xuất .']

In [16]:
manual_tokenize_compounds = []
for sentence in manual_tokenize_sentences:
    for word in sentence.split():
        if '_' in word: manual_tokenize_compounds.append(word.lower())
print('Số lượng từ ghép khi tách từ thủ công:', len(manual_tokenize_compounds))

Số lượng từ ghép khi tách từ thủ công: 309


### So sánh kết quả

In [18]:
# liệt kê các từ ghép sai
wrong_compounds = []
for word in longest_matching_compounds:
    if word not in manual_tokenize_compounds: wrong_compounds.append(word)
wrong_compounds = list(set(wrong_compounds))
print('Số lượng từ ghép không có khi làm thủ công:', len(wrong_compounds))
print('Cụ thể:', wrong_compounds)

# liệt kê các từ không có
missing_compounds = []
for word in manual_tokenize_compounds:
    if word not in longest_matching_compounds: missing_compounds.append(word)
    # if word.replace('_',' ') not in vocab: print(word.replace('_',' '))
missing_compounds = list(set(missing_compounds))
print('Số lượng từ ghép không có:', len(missing_compounds))
print('Cụ thể:', missing_compounds)

Số lượng từ ghép không có khi làm thủ công: 73
Cụ thể: ['thấy_rõ', 'tháng_12', 'gây_nên', 'đầu_người', 'bắn_lên', 'xảy_ra', 'dẫn_đến', 'nguồn_tin', 'trong_khi', 'giải_đấu', 'đưa_ra', 'xem_là', 'nhà_toán_học', 'gặp_trục_trặc', 'bên_phải', 'dành_thời_gian', 'cho_biết', 'lớn_lên', 'bảng_xếp_hạng', 'bị_hỏng', 'bóng_đá', 'món_ăn', 'người_trước', 'năm_nay', 'người_uống', 'trở_về', 'dòng_máu', 'năm_trường', 'biết_trước', 'vũ_khí_hạt_nhân', 'tăng_đột_biến', 'nói_là', 'gây_tắc', 'hiện_đại', 'đại_diện_cho', 'có_thói_quen', 'bị_tắc', 'người_đến', 'cũng_vậy', 'chiến_đấu_với', 'vận_chuyển', 'tưởng_rằng', 'xe_rác', 'ở_chỗ', 'chống_lại', 'mỗi_ngày', 'cho_đang', 'thứ_2', 'bằng_chứng_ngoại_phạm', 'kết_quả_là', 'được_trang_bị', 'cao_hơn', 'thiếu_máu', 'có_tài', 'diễn_ra', 'thời_gian_đầu', 'có_dấu_hiệu', 'không_vững', 'quả_bóng', 'người_lao_động', 'nhà_cao_tầng', 'trụ_được', 'không_có', 'mạnh_mẽ_hơn', 'hàng_không', 'tốt_hơn', 'gần_đây', 'lý_trường', 'bức_tranh', 'não_giữa', 'cao_nhất', 'có_được', 'nữ_côn